In [1]:
print("hello world")

hello world


Okay so we have to write a custom dataset class no biggie, but the project description says to focus on silence and unknown but there is no such labels 

I guess for unknown you can make argument that special words like marvin, sheila, happy can serve as unknown but not sure about silence.

Okay case closed we are supposed to create it ourselves based on background_noise_ directory 

Goal for this notebook is to prepare data. I'm assuming that in /data
directory should be extracted contents of train archive:

- `audio` directory 
- testing_list.txt
- validation_list.txt

I'm gonna create three directories in /data

- `train`
- `test`
- `validation`

im gonna add new class to each: silence

In [ ]:
from pathlib import Path
from typing import Tuple

import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio

In [2]:
class SpeechCommandsDataset(Dataset):
    def __init__(self, root_dir: str, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform

        # Get all .wav file paths and corresponding labels
        self.samples = []
        self.labels = sorted({p.name for p in self.root_dir.iterdir() if p.is_dir()})
        self.label_to_index = {label: idx for idx, label in enumerate(self.labels)}

        for label in self.labels:
            label_dir = self.root_dir / label
            for wav_file in label_dir.glob("**/*.wav"):
                self.samples.append((wav_file, self.label_to_index[label]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx) -> Tuple[torch.Tensor, int]:
        audio_path, label = self.samples[idx]
        waveform, sample_rate = torchaudio.load(audio_path)

        # Optional transformation (e.g., resample, MFCC, etc.)
        if self.transform:
            waveform = self.transform(waveform)

        return waveform, label

In [7]:
train_dataset = SpeechCommandsDataset("./../../data/train")
val_dataset = SpeechCommandsDataset("./../../data/validation")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Example: get a batch
for batch in train_loader:
    waveforms, labels = batch
    print(waveforms.shape)  # (batch_size, channels, samples)
    print(labels)
    break

RuntimeError: stack expects each tensor to be equal size, but got [1, 16000] at entry 0 and [1, 12288] at entry 9

In [8]:
import os
from torch.utils.data import Dataset, DataLoader
import torchaudio


class AudioDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Scan directories and store paths and labels
        for label in os.listdir(root_dir):
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                for fname in os.listdir(label_dir):
                    if fname.endswith(".wav"):
                        file_path = os.path.join(label_dir, fname)
                        self.samples.append((file_path, label))

        # Get sorted list of labels and map to index
        self.label_to_index = {
            label: idx
            for idx, label in enumerate(sorted(set(label for _, label in self.samples)))
        }

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        waveform, sample_rate = torchaudio.load(file_path)

        if self.transform:
            waveform = self.transform(waveform)

        label_idx = self.label_to_index[label]
        return waveform, label_idx

In [12]:
def collate_fn(batch):
    waveforms, labels = zip(*batch)
    return list(waveforms), torch.tensor(labels)

In [18]:
dataset = AudioDataset(root_dir="./../../data/train")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

for batch_idx, (waveforms, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx}")
    print(f"Waveforms shape: {waveforms}")
    print(f"Labels: {labels}")
    break

Batch 0
Waveforms shape: [tensor([[0.0000e+00, 3.0518e-05, 9.1553e-05,  ..., 3.0518e-05, 6.1035e-05,
         6.1035e-05]]), tensor([[-0.0006, -0.0006, -0.0006,  ...,  0.0007,  0.0007,  0.0006]]), tensor([[ 0.0000e+00, -6.1035e-05, -6.1035e-05,  ..., -6.1035e-05,
          0.0000e+00, -9.1553e-05]]), tensor([[0.0000e+00, 3.0518e-05, 9.1553e-05,  ..., 6.1035e-05, 3.0518e-05,
         6.1035e-05]])]
Labels: tensor([ 1, 24, 21, 21])
